In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk   
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
from nltk import PorterStemmer, WordNetLemmatizer 

import string
import collections

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\Yunsun2\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/yelp_cleaned.csv', encoding='latin-1', index_col=0)

In [3]:
#Sentiment analysis;
def sentiment_score(text):
    neg_words={'awkwardly', 'awkward', 'butt', 'discount', 'disappointed', 'disappoint', 
               'dismay', 'dumb', 'manner', 'sue', 'mean', 'meeting', 'monte', 'unfriendly', 'nasty', 'refuse', 
               'avoid', 'soggy', 'mistake', 'upset', 'filthy', 'sad', 'lousy', 'worst',  'unprofessional', 
               'rudely','unfortunately', 'screw', 'poorly', 'crap', 'crappy', 'tasteless', 'ignore', 'disaster',
               'overpriced', 'wrong', 'sick', 'inedible', 'stale', 'dirty', 'disappointment', 'sorry', 'yuck', 
               'flavorless', 'mess', 'dont', 'slow', 'lack', 'wouldnt', 'disappointed', 'terrible', 'overprice', 
               'gross', 'waste', 'poor', 'leave', 'disgust', 'awful', 'mediocre', 'suck', 'bland', 'wasnt', 
               'rude', 'didnt', 'horrible', 'bad'}
                   
    pos_words={'ambiance', 'ambience', 'awhile', 'delight', 'generous', 'hilarious', 'homey', 'hometown', 'home'
               'performance', 'performer', 'recover', 'tremendous', 'fantastically', 'fantastic', 'friendly', 'funny', 
               'goodwill', 'successfully', 'amaze', 'amazing', 'amazingly', 'magic', 'favourite', 'favorite', 'extraordinarily'
               'great', 'love', 'best', 'delicious', 'perfect', 'awesome', 'recommend', 'excellent', 'highly', 'fresh', 
               'friendly', 'wonderful', 'perfectly', 'like', 'beautiful', 'fabulous', 'cool', 'super', 'heaven', 'incredible'}   
    
    score=0
    for i in str(text).split():
        if i in pos_words:
            score+=1
        elif i in neg_words:
            score-=1
    return score
    
data['sentiment']=data['textc'].apply(sentiment_score)
print(data.corr())  #sentiment now is 0.49 corr with stars; 

              stars      cool    useful     funny  text_len  sentiment
stars      1.000000  0.052555 -0.023479 -0.061306 -0.092193   0.492147
cool       0.052555  1.000000  0.887102  0.764342  0.237975   0.060880
useful    -0.023479  0.887102  1.000000  0.723406  0.290306   0.028723
funny     -0.061306  0.764342  0.723406  1.000000  0.242552  -0.022380
text_len  -0.092193  0.237975  0.290306  0.242552  1.000000   0.063492
sentiment  0.492147  0.060880  0.028723 -0.022380  0.063492   1.000000


In [4]:
data['stars'].value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [5]:
#re-value stars 4 & 5
data['star']=data['stars'].apply(lambda x: 5 if x>=4 else x)

In [13]:
data.drop('stars', axis=1, inplace=True)

In [17]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Z=data.drop('textc', axis=1)
X=data['textc'] #will add other features back later on for another analysis;   

rs=3746
x_train, x_test, ztrain, ztest = train_test_split(X, Z, test_size=0.3, random_state=rs) 

#TfidfVectorizer
tfidfv=TfidfVectorizer(min_df=2, ngram_range=(1, 1), stop_words='english', max_features=10000, strip_accents='unicode', 
                           norm='l2')

xtrain=tfidfv.fit_transform(x_train.values.astype('U')).todense()
xtest=tfidfv.transform(x_test.values.astype('U')).todense()
col = ['feat_'+i for i in tfidfv.get_feature_names()]  #for TfidfVectorizer

_xtrain = pd.DataFrame(xtrain, columns=col)  #feature names added
_xtest = pd.DataFrame(xtest, columns=col)  #row index from 0 to 6999 (number of reviews);  

temptrain=ztrain.loc[:, ['cool', 'useful', 'funny', 'text_len', 'sentiment']].reset_index(drop=True)  #remove 'sentiment' needed;
trainfull=pd.concat([temptrain, _xtrain], axis=1)

temptest=ztest.loc[:, ['cool', 'useful', 'funny', 'text_len', 'sentiment']].reset_index(drop=True) #remove 'sentiment' needed;
testfull=pd.concat([temptest, _xtest], axis=1)

ytrain=ztrain['star']
ytest=ztest['star']

In [18]:
from sklearn.ensemble import RandomForestRegressor

result=pd.DataFrame()
rs=1928
global result

'''
model=RandomForestRegressor(random_state=rs)
cname='RandomForestRegressor'

model=XGBClassifier(random_state=rs)
cname='XGBoost'
'''
model=LogisticRegression(random_state=rs)
cname='LogisticRegression'

model.fit(trainfull, ytrain)
pred=model.predict(testfull)
tpred=model.predict(trainfull)
    
#RFRegressor has not attributes of predict_proba; 
predproba=pd.DataFrame(model.predict_proba(testfull))
tpredproba=pd.DataFrame(model.predict_proba(trainfull))
#merge with test and train labels; 

print(cname + ' predictions with default hyparameters for Test'+'\n')
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))  

print(cname + 'predictions with default hyparameters for Train'+'\n')
print('Accuracy')
print(accuracy_score(ytrain, tpred, normalize=True))

#compare bad predictions of different models; 
result=pd.concat([result, pd.DataFrame(pred)], axis=1) #to merge the predicted labels back to the dataset;
result.rename(columns={0:str(cname + '_pred')}, inplace=True)

LogisticRegression predictions with default hyparameters for Test

Accuracy
0.718666666667
[[  71   33   13  112]
 [  34   43   25  167]
 [  15   25   56  381]
 [   8    5   26 1986]]
             precision    recall  f1-score   support

          1       0.55      0.31      0.40       229
          2       0.41      0.16      0.23       269
          3       0.47      0.12      0.19       477
          5       0.75      0.98      0.85      2025

avg / total       0.66      0.72      0.65      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.804714285714


In [19]:
from sklearn.ensemble import RandomForestRegressor

result=pd.DataFrame()
rs=300
global result

'''
model=RandomForestRegressor(random_state=rs)
cname='RandomForestRegressor'

model=XGBClassifier(random_state=rs)
cname='XGBoost'
'''
model=LogisticRegression(random_state=rs)
cname='LogisticRegression'

model.fit(trainfull, ytrain)
pred=model.predict(testfull)
tpred=model.predict(trainfull)
    
#RFRegressor has not attributes of predict_proba; 
predproba=pd.DataFrame(model.predict_proba(testfull))
tpredproba=pd.DataFrame(model.predict_proba(trainfull))
#merge with test and train labels; 

print(cname + ' predictions with default hyparameters for Test'+'\n')
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))  

print(cname + 'predictions with default hyparameters for Train'+'\n')
print('Accuracy')
print(accuracy_score(ytrain, tpred, normalize=True))

#compare bad predictions of different models; 
result=pd.concat([result, pd.DataFrame(pred)], axis=1) #to merge the predicted labels back to the dataset;
result.rename(columns={0:str(cname + '_pred')}, inplace=True)

LogisticRegression predictions with default hyparameters for Test

Accuracy
0.718666666667
[[  71   33   13  112]
 [  34   43   25  167]
 [  15   25   56  381]
 [   8    5   26 1986]]
             precision    recall  f1-score   support

          1       0.55      0.31      0.40       229
          2       0.41      0.16      0.23       269
          3       0.47      0.12      0.19       477
          5       0.75      0.98      0.85      2025

avg / total       0.66      0.72      0.65      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.804714285714


In [ ]:
#Much better now! Star 5 and Star 1 greatly improved. 
#Can do more sentiment/error analysis to see why star 103 has been much mis-classified to 4/5 (5 now)